<a href="https://colab.research.google.com/github/nikopensius/rl4kgReasoning/blob/main/proofOfConcept.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ampligraph
!pip uninstall tensorflow
!pip install tensorflow==1.15

In [ ]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/thesis/deeppath/FB15k-237/tasks/Combined')

In [ ]:
import pandas as pd
import pickle
filepath = "action_space.pkl"
action_space = pickle.load(open(filepath, "rb"))
print("Length of action_space pre FOBO removal:", len(action_space))
test = pd.read_csv("test.csv").drop("Unnamed: 0", axis=1)
FOBO = list(test.rr.unique())
FOBO_inv = [relation + "_inv" for relation in FOBO]
FOBO.extend(FOBO_inv)

action_space = [action for action in action_space if action not in FOBO]
print("Length of action_space post FOBO removal:", len(action_space))
print("Saving updated action_space into", filepath)
pickle.dump(action_space, open(filepath, "wb"))

Length of action_space pre FOBO removal: 324
Length of action_space post FOBO removal: 318
Saving updated action_space into action_space.pkl


In [ ]:
import pickle
import pandas as pd
import numpy as np
import ampligraph
from ampligraph.latent_features import restore_model
import copy
"""
from google.colab import drive
drive.mount('/content/drive')
"""
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/thesis/deeppath/NELL-995/tasks/")
"""
os.chdir('/content/drive/My Drive/Colab Notebooks/thesis/deeppath/FB15k-237/tasks')

### Read triples and pairs ###
load = 1

go_to = 36 #0-34
dir = os.listdir()
os.chdir('./' + dir[go_to])
!pwd
"""
dir = os.listdir()
for i, f in enumerate(dir):
  print(i, f)
go_to = int(input())
os.chdir('./' + dir[go_to])

load = 1
if not load:
  kg_triples = read_kg_triples()  # used by agent to look for steps in path
  pickle.dump(kg_triples, open("kg_triples.pkl", "wb"))
  input = read_input()    # split into train and test set
  input.to_csv("input.csv")
  df_kg = read_triples()  # used for training KG
  df_kg.to_csv("df_kg")
  train_pos = train_pos()
  pickle.dump(train_pos, open("train_pos.pkl", "wb"))

0 concept_athleteplayssport
1 concept_agentbelongstoorganization
2 concept_organizationheadquarteredincity
3 concept_athleteplaysinleague
4 concept_athleteplaysforteam
5 concept_athletehomestadium
6 concept_teamplaysinleague
7 concept_personleadsorganization
8 concept_teamplayssport
9 concept_organizationhiredperson
10 concept_worksfor
11 concept_personborninlocation
12 application.log
13 Combined
10


In [ ]:
kg_triples = pickle.load(open("kg_triples.pkl", "rb"))
inputt = pd.read_csv("input.csv")

for index, data in inputt.iterrows():
  pair = data.es + data.rr
  if pair in kg_triples:
    del kg_triples[pair]

to_del = []
for key in kg_triples:
  if ("type_of_union" in key) or ("category" in key):
    to_del.append(key)

for key in to_del:
  del kg_triples[key]

pickle.dump(kg_triples, open("kg_triples.pkl", "wb"))

##REINFORCEMENT LEARNING

In [ ]:
import torch  
import gym
import numpy as np  
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt
import random

# Constants
GAMMA = 0.95 ###################################################################################

class PolicyNetwork(nn.Module):
    def __init__(self, num_inputs, num_actions, hidden_size, learning_rate=1e-5, manual_seed = 0):
        if manual_seed:      # set seed manually for controlled experimentla environment
          torch.manual_seed(manual_seed)
          np.random.seed(manual_seed)
        super(PolicyNetwork, self).__init__()
        self.train = True
        self.num_actions = num_actions
        self.linear1 = nn.Linear(num_inputs, hidden_size) #####################################
        self.linear2 = nn.Linear(hidden_size, num_actions) ####################################
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate) ######################
        self.illegal_ent = "/m/08mbj5d"

    def forward(self, state):
        x = F.relu(self.linear1(state.flatten()))
        x = F.softmax(self.linear2(x))
        return x
    
    def get_val_action(self, probs, action_space, current_ent, kg):
        while True:
          # get index of most probable action = maxProb_i,
          maxProb_i = probs.argmax()

          # retrieve most probable action
          action = action_space[maxProb_i]

          # check if most probable action leads to a new entity
          key = current_ent + action

          if key in kg and kg[key] != [self.illegal_ent]:
              return maxProb_i
          else:
            # disqualify the maximum probability from further choice
            probs[maxProb_i] = -1
    

    #"""
    # get_val_actions - a variation for BEAM search
    def get_val_actions(self, probs, action_space, current_ent, kg, b):
        max_is = []
        for _ in range(len(probs)):
          # get index of most probable action = maxProb_i,
          maxProb_i = probs.argmax()

          # retrieve most probable action
          action = action_space[maxProb_i]

          # check if most probable action leads to a new entity
          key = current_ent + action
          if key in kg and kg[key] != [self.illegal_ent]:
              max_is.append(maxProb_i)
          # disqualify the maximum probability from further choice
          probs[maxProb_i] = -1
          
          if len(max_is) == b:
            break
        #if not max_is:
          #print(current_ent)
        return max_is

    def get_actions(self, state, action_space, current_ent, kg_triples, b=1):
        # Convert state to tensor
        state = torch.from_numpy(state).float().unsqueeze(0)
        # Calculate probabilities over action space
        probs = self.forward(Variable(state))
        probs_array = probs.detach().numpy()
        copy_of_probs = copy.deepcopy(probs_array)  #to be used in highest_prob_action
                                                    #the problem is, the thing will not know that its top probabilities lead to nowhere                                              

        highest_prob_actions = self.get_val_actions(copy_of_probs, action_space, current_ent, kg_triples, b)
      
        jou = probs.squeeze(0).squeeze(0)
        log_probs = []
        for hpa in highest_prob_actions:
          joujou = jou[hpa]
          log_prob = torch.log(joujou)
          log_probs.append(log_prob)
        return highest_prob_actions, log_probs
            
    #"""


    def get_random_val_action(self, probs, action_space, current_ent, kg):
        no_good = []
        while True:
          # get index of random action
          randProb_i = self.random_arg(action_space, probs)
          if randProb_i in no_good:
            continue

          # retrieve the action
          action = action_space[randProb_i]

          # check if the action leads to a new entity
          key = current_ent + action
          if key in kg and kg[key] != [self.illegal_ent]:
            return randProb_i
          else:
            no_good.append(randProb_i)

    def random_arg(self, action_space, action_probs):
      return np.random.choice(np.arange(len(action_space)), p = np.squeeze(action_probs))

    def get_random_val_action(self, probs, action_space, current_ent, kg):
        while True:
          # get index of random action
          randProb_i = self.random_arg(action_space, probs)

          # retrieve the action
          action = action_space[randProb_i]

          # check if the action leads to a new entity
          key = current_ent + action
          if kg[key] != [self.illegal_ent]:
            return randProb_i
    
    def mask_probs(self, action_space, current_ent, kg_triples, probs):
        for i, action in enumerate(action_space):
          key = current_ent + action
          if key not in kg_triples:
            probs[i] = 0
        return probs / sum(probs)
    
    def normal_distribution(self, highest_prob_actions, probs):
        highest_probs = []
        for hpi in highest_prob_actions:
          highest_probs.append(probs[hpi])
        highest_probs = np.asarray(highest_probs)
        highest_probs = highest_probs / sum(highest_probs)
        return highest_probs

    def get_action(self, state, action_space, current_ent, kg_triples):
        # Convert state to tensor
        state = torch.from_numpy(state).float().unsqueeze(0)
        # Calculate probabilities over action space
        probs = self.forward(Variable(state))
        probs_array = probs.detach().numpy()
        copy_of_probs = copy.deepcopy(probs_array)  #to be used in highest_prob_action
                                                    #the problem is, the thing will not know that its top probabilities lead to nowhere
        
        rand = False
        if rand:
          masked_copy_of_probs = self.mask_probs(action_space, current_ent, kg_triples, copy_of_probs)
          highest_prob_action = self.get_random_val_action(masked_copy_of_probs, action_space, current_ent, kg_triples)#.item()
        else:
          #highest_prob_action = self.get_val_action(copy_of_probs, action_space, current_ent, kg_triples)
          highest_prob_actions = self.get_val_actions(copy_of_probs, action_space, current_ent, kg_triples, 1) #NELL-995 change
          if not highest_prob_actions:
            print(current_ent)
          distribution = self.normal_distribution(highest_prob_actions, copy.deepcopy(probs_array))
          highest_prob_action = highest_prob_actions[np.random.choice(np.arange(len(highest_prob_actions)), p = distribution)]
      
        jou = probs.squeeze(0).squeeze(0)
        joujou = jou[highest_prob_action]
        log_prob = torch.log(joujou)
        return highest_prob_action, log_prob




def update_policy(policy_network, rewards, log_probs):
    discounted_rewards = []

    for t in range(len(rewards)):
        Gt = 0 
        pw = 0
        for r in rewards[t:]:
            Gt = Gt + GAMMA**pw * r
            pw = pw + 1
        discounted_rewards.append(Gt)
        
    discounted_rewards = torch.tensor(discounted_rewards)
    discounted_rewards = (discounted_rewards - discounted_rewards.mean()) / (discounted_rewards.std() + 1e-9) # normalize discounted rewards
                                                                                                        ##################################

    policy_gradient = []
    for log_prob, Gt in zip(log_probs, discounted_rewards):
        policy_gradient.append(-log_prob * Gt)
    torch.autograd.set_detect_anomaly(True)
    policy_network.optimizer.zero_grad()
    policy_gradient = torch.stack(policy_gradient).sum()
    policy_gradient.backward()
    policy_network.optimizer.step()





from numpy import dot
from numpy.linalg import norm


# Use this
import pickle
import random
from ampligraph.latent_features import restore_model

class Environment_train():
  
  def __init__(self, path_to_dataset, path_to_model):
    self.model_name   = path_to_model
    self.i            = 2
    self.states       = None
    self.train_pos    = pickle.load(open("train_pos.pkl", "rb"))
    print(os.listdir())
    self.action_space = pickle.load(open("action_space.pkl", "rb"))
    self.action_space.append("selfLoop")
    self.input        = pd.read_csv(path_to_dataset)
    self.model        = restore_model(path_to_model)          # This is a KG model
    self.kg           = pickle.load(open("kg_triples.pkl", "rb"))
    self.stats        = {"Set size:": self.input.shape[0],
                         "Size of action space:" : 0,
                         "Number of episodes:": 0,                          # env.stats["Number of episodes:"]
                         "Target found" : 0}
    n_input, n_hidden, n_out, batch_size, learning_rate = 440, 300, 1, 1, 0.01

    self.classification_model = nn.Sequential(nn.Linear(n_input, n_hidden),
                      nn.ReLU(),
                      nn.Linear(n_hidden, n_out),
                      nn.Sigmoid())
    self.loss_function = nn.MSELoss()
    self.optimizer = torch.optim.SGD(self.classification_model.parameters(), lr=learning_rate)

  def __get_state(self, i):
    if type(i) != int:
      i = random.randint(0, self.input.shape[0] - 1)
    row = self.input.iloc[i]
    triple = [row.et, row.rr, row.es]
    #FB15K-237
    #self.actual_target = self.train_pos[row.es]
    #NELL-995
    self.actual_target = self.train_pos[row.es+row.rr]
    self.source = row.es
    self.relation = row.rr
    self.claimed_target = row.et
    hr_state = triple + [0, 0, 0, 0, 0, 0, 0, 0]
    y = row.y

    source_and_target_embeddings = self.model.get_embeddings([triple[0], triple[2]])
    relation_embedding = self.model.get_embeddings([triple[1]], "relation")[0]
    state = np.zeros((11, 40))
    state[0] = source_and_target_embeddings[0]
    state[1] = relation_embedding
    state[2] = source_and_target_embeddings[1]
    
    return [hr_state], [state], y

  def reset(self, i = None):
    self.hr_states, self.states, self.y = self.__get_state(i)      # this is a list
    self.i = 2
    self.complex_source = self.complex_vector(self.source)
    self.complex_relation = self.complex_vector(self.relation)
    return self.hr_states, self.states

  def __get_embedding(self, entity_ID, emb_type="entity"):
    #print(entity_ID)
    return self.model.get_embeddings([entity_ID], emb_type)


  def complex_vector(self, string):
    object_type = "relation"
    #if string[0] == "/":
    try:
      test_coords = string.split(",")
      for el in test_coords:
        float(el)
      object_type = "entity"
    except:
      if ("_" in string):
        object_type = "entity"
    try: embedding = self.model.get_embeddings([string], object_type)[0]
    except:
      print(string, object_type)
    return self.complexify(embedding)

  def dot_product(self, candidate):

    source = self.complex_source
    relation = self.complex_relation
    candidate = self.complex_vector(candidate).conjugate()

    dot_product = 0
    for u, a, e in zip(source, relation, candidate):
      dot_product += u * a * e
    return dot_product.real

  def complexify(self, input_vector):
    output_vector = []
    for real, imag in zip(input_vector[:20], input_vector[20:]):
      comp = complex(real, imag)
      output_vector.append(comp)
    output_vector = np.asarray(output_vector)
    return output_vector

  def __cosine_similarity(self, candidate):

    source = self.model.get_embeddings([self.source])[0]
    relation = self.model.get_embeddings([self.relation], "relation")[0]
    target = source + relation
    
    return dot(candidate, target) / (norm(candidate)*norm(target))

# 21.08.2022
# Experiment no 2:
#  ComplEx embedding with entity choice cos(claimed target, candidate)

  def __cosine_similarity(self, candidate):

    target = self.model.get_embeddings([self.claimed_target])[0]
    
    return dot(candidate, target) / (norm(candidate)*norm(target))
  
  def query(self, entity, action):
    key = entity + action
    return self.kg[key]
  
  def extend_state(self, hr_state, state, action):
    state = copy.deepcopy(state)
    hr_state = copy.deepcopy(hr_state)
    current_ent = hr_state[self.i]
    new_ent_candidates = self.query(current_ent, action)
    new_ent, new_ent_embed = self.best_candidate(new_ent_candidates)

    hr_state[self.i + 1] = action
    hr_state[self.i + 2] = new_ent
    state[self.i + 1] = self.__get_embedding(action, "relation")[0]
    state[self.i + 2] = new_ent_embed

    return hr_state, state, new_ent

    
  def sigmoid(self, x):
    return 1 / (1 + np.exp(-x))

  def best_candidate(self, candidates):
    if "/m/08mbj5d" in candidates:
      candidates.remove("/m/08mbj5d")
    candidates_embedded = self.model.get_embeddings(candidates)
    if self.model_name == "TransE":
      similarities = map(self.__cosine_similarity, candidates_embedded)
      similarities = np.asarray(list(similarities))
      index_of_most_similar = similarities.argmax()
    elif self.model_name == "ComplEx":
      random = False
      if random:
        dot_products = map(self.dot_product, candidates)
        dot_products = np.asarray(list(dot_products))
        sigmoid_same = map(self.sigmoid, dot_products)
        sigmoid_same = np.asarray(list(sigmoid_same))
        scores = []
        indices = []
        for _ in range(3):
          index_of_most_similar = sigmoid_same.argmax()
          indices.append(sigmoid_same.argmax())
          scores.append(sigmoid_same.max())
          sigmoid_same[index_of_most_similar] = 0
        scores = np.asarray(scores)
        scores /= sum(scores)
        index_of_most_similar = np.random.choice(indices, p=scores)
        new_ent = candidates[index_of_most_similar]
        new_ent_embedded = candidates_embedded[index_of_most_similar]
      else:
        dot_products = map(self.dot_product, candidates)
        dot_products = np.asarray(list(dot_products))
        index_of_most_similar = dot_products.argmax()
    new_ent = candidates[index_of_most_similar]
    new_ent_embedded = candidates_embedded[index_of_most_similar]
    return new_ent, new_ent_embedded

  """
# 21.08.2022
# Experiment no 1:
#  ComplEx embedding with entity choice cos(TransE scoring function based virtual target, candidate)
#  yields a high precision because ComplEx is a good embedding
#  and the TransE

  def best_candidate(self, candidates):
    if "/m/08mbj5d" in candidates:
      candidates.remove(policy_net.illegal_ent)
    candidates_embedded = self.model.get_embeddings(candidates)
    similarities = map(self.__cosine_similarity, candidates_embedded)
    similarities = np.asarray(list(similarities))
    index_of_most_similar = similarities.argmax()
    new_ent = candidates[index_of_most_similar]
    new_ent_embedded = candidates_embedded[index_of_most_similar]
    return new_ent, new_ent_embedded
  """
  def step(self, hr_state, state, action_index):   #BRING IN THE STATE

    reward = 0
    
    action = self.action_space[action_index]
    new_hr_state, new_state, new_ent = self.extend_state(hr_state, state, action)

    if new_ent in self.actual_target: # BLIND CHANGE
      reward = 1

    self.states[0][self.i + 1] = self.model.get_embeddings([action], "relation")
    self.hr_states[0][self.i + 1] = action

    self.states[0][self.i + 2] = self.model.get_embeddings([new_ent])[0]
    self.hr_states[0][self.i + 2] = new_ent
    
    return new_hr_state, new_state, reward #for new_state, return and embedding of the entity (which is the state)






import numpy as np
from numpy import dot
from numpy.linalg import norm

# Use this
import pickle
import random
from ampligraph.latent_features import restore_model

class Environment_test():
  
  def __init__(self, path_to_dataset, path_to_model):
    self.model_name   = path_to_model
    self.i            = 2
    self.states       = None
    self.train_pos    = pickle.load(open("train_pos.pkl", "rb"))
    self.action_space = pickle.load(open("action_space.pkl", "rb"))
    self.action_space.append("selfLoop")
    self.input        = pd.read_csv(path_to_dataset).sample(frac=0.3, random_state=1)
    self.model        = restore_model(path_to_model)          # This is a KG model
    self.kg           = pickle.load(open("kg_triples.pkl", "rb"))
    self.stats        = {"Set size:": self.input.shape[0],
                         "Size of action space:" : 0,
                         "Number of episodes:": 0,                          # env.stats["Number of episodes:"]
                         "Target found" : 0}
    n_input, n_hidden, n_out, batch_size, learning_rate = 440, 300, 1, 1, 0.01

    self.classification_model = nn.Sequential(nn.Linear(n_input, n_hidden),
                      nn.ReLU(),
                      nn.Linear(n_hidden, n_out),
                      nn.Sigmoid())
    self.loss_function = nn.MSELoss()
    self.optimizer = torch.optim.SGD(self.classification_model.parameters(), lr=learning_rate)

  def __get_state(self, i):
    if type(i) != int:
      i = random.randint(0, self.input.shape[0] - 1)
    row = self.input.iloc[i]
    triple = [row.et, row.rr, row.es]
    # FB15K-237
    #self.actual_target = self.train_pos[row.es]
    # NELL-995
    self.actual_target = self.train_pos[row.es + row.rr]
    self.source = row.es
    self.relation = row.rr
    self.claimed_target = row.et
    hr_state = triple + [0, 0, 0, 0, 0, 0, 0, 0]
    y = row.y

    source_and_target_embeddings = self.model.get_embeddings([triple[0], triple[2]])
    relation_embedding = self.model.get_embeddings([triple[1]], "relation")[0]
    state = np.zeros((11, 40))
    state[0] = source_and_target_embeddings[0]
    state[1] = relation_embedding
    state[2] = source_and_target_embeddings[1]
    
    return [hr_state], [state], y

  def reset(self, i = None):
    self.hr_states, self.states, self.y = self.__get_state(i)      # this is a list
    self.i = 2
    self.complex_source = self.complex_vector(self.source)
    self.complex_relation = self.complex_vector(self.relation)
    return self.hr_states, self.states

  def __get_embedding(self, entity_ID, emb_type="entity"):
    return self.model.get_embeddings([entity_ID], emb_type)

  def complex_vector(self, string):
    object_type = "relation"
    #if string[0] == "/":
    try:
      test_coords = string.split(",")
      for el in test_coords:
        float(el)
      object_type = "entity"
    except:
      if ("_" in string):
        object_type = "entity"
    embedding = self.model.get_embeddings([string], object_type)[0]
    return self.complexify(embedding)

  def dot_product(self, candidate):

    source = self.complex_source
    relation = self.complex_relation
    candidate = self.complex_vector(candidate).conjugate()

    dot_product = 0
    for u, a, e in zip(source, relation, candidate):
      dot_product += u * a * e
    return dot_product.real

  def complexify(self, input_vector):
    output_vector = []
    for real, imag in zip(input_vector[:20], input_vector[20:]):
      comp = complex(real, imag)
      output_vector.append(comp)
    output_vector = np.asarray(output_vector)
    return output_vector

  def __cosine_similarity(self, candidate):

    source = self.model.get_embeddings([self.source])[0]
    relation = self.model.get_embeddings([self.relation], "relation")[0]
    target = source + relation
    
    return dot(candidate, target) / (norm(candidate)*norm(target))

# 21.08.2022
# Experiment no 2:
#  ComplEx embedding with entity choice cos(claimed target, candidate)

  def __cosine_similarity(self, candidate):

    target = self.model.get_embeddings([self.claimed_target])[0]
    
    return dot(candidate, target) / (norm(candidate)*norm(target))
  
  def query(self, entity, action):
    key = entity + action
    return self.kg[key]
  
  def extend_state(self, hr_state, state, action, b):
    state = copy.deepcopy(state)
    hr_state = copy.deepcopy(hr_state)
    current_ent = hr_state[self.i]
    new_ent_candidates = self.query(current_ent, action)
    new_ents, new_ents_embed, scores = self.best_candidate(new_ent_candidates, b)

    hr_state[self.i + 1] = action
    state[self.i + 1] = self.__get_embedding(action, "relation")[0]

    hr_states = []
    states = []
    for i in range(len(new_ents)):
      h = copy.deepcopy(hr_state)
      h[self.i + 2] = new_ents[i]
      #h = np.asarray(h)
      hr_states.append(h)
      s = copy.deepcopy(state)
      #s = np.asarray(s)
      s[self.i + 2] = new_ents_embed[i]
      states.append(s)

    return hr_states, states, new_ents, scores

  def sigmoid(self, x):
    return 1 / (1 + np.exp(-x))

  def best_candidate(self, candidates, b):
    if "/m/08mbj5d" in candidates:
      candidates.remove("/m/08mbj5d")
    candidates_embedded = self.model.get_embeddings(candidates)
    if self.model_name == "TransE":
      similarities = map(self.__cosine_similarity, candidates_embedded)
      similarities = np.asarray(list(similarities))
      index_of_most_similar = similarities.argmax()
    elif self.model_name == "ComplEx":
      dot_products = map(self.dot_product, candidates)
      dot_products = np.asarray(list(dot_products))
      sigmoid_same = map(self.sigmoid, dot_products)
      sigmoid_same = np.asarray(list(sigmoid_same))
      new_ents = []
      new_ents_embedded = []
      scores = []
      for _ in range(b):
        index_of_most_similar = sigmoid_same.argmax()
        scores.append(sigmoid_same.max())
        sigmoid_same[index_of_most_similar] = 0
        new_ent = candidates[index_of_most_similar]
        new_ent_embedded = candidates_embedded[index_of_most_similar]
        new_ents.append(new_ent)
        new_ents_embedded.append(new_ent_embedded)
    return new_ents, new_ents_embedded, scores
  """
# 21.08.2022
# Experiment no 1:
#  ComplEx embedding with entity choice cos(TransE scoring function based virtual target, candidate)
#  yields a high precision because ComplEx is a good embedding
#  and the TransE

  def best_candidate(self, candidates):
    if "/m/08mbj5d" in candidates:
      candidates.remove(policy_net.illegal_ent)
    candidates_embedded = self.model.get_embeddings(candidates)
    similarities = map(self.__cosine_similarity, candidates_embedded)
    similarities = np.asarray(list(similarities))
    index_of_most_similar = similarities.argmax()
    new_ent = candidates[index_of_most_similar]
    new_ent_embedded = candidates_embedded[index_of_most_similar]
    return new_ent, new_ent_embedded
  """
  def step(self, hr_state, state, action_index):   #BRING IN THE STATE

    reward = 0
    
    action = self.action_space[action_index]
    new_hr_states, new_states, new_ents, scores = self.extend_state(hr_state, state, action, b)

    for new_ent in new_ents:
      if new_ent in self.actual_target: # BLIND CHANGE
        reward = 1
    
    return new_hr_states, new_states, reward, new_ents, scores #for new_state, return and embedding of the entity (which is the state)





import random

def train_model(path_to_model, max_episode_num, max_steps, random_state = 0, manual_seed = 0, policy_net = None, start = 0):
    # Maybe these two should be chosen from a random title?
    # Or maybe another agent should choose them?
    # How should it learn to choose?
    #   Maybe using embeddings to decide(learn)?

    size_observation_space = 440 #(len(list(relevante_aliases.keys())))
    env = Environment_train("train.csv", path_to_model)
    size_action_space = len(env.action_space)
    if not policy_net:
      policy_net = PolicyNetwork(size_observation_space, size_action_space, 128, manual_seed = manual_seed)
    
    len_dataset = env.input.shape[0]
    env.stats["Number of episodes:"] = max_episode_num
    env.stats["Size of action space:"] = size_action_space
    max_steps = max_steps
    all_rewards = []
    avg_rewards = []

    # collect confidence scores to calculate confidence thresholds for true and fake classifications
    confidence_scores = {"true_closed"  : [], # y = +, hr[0] == hr[-1]
                         "fake_closed"  : [], # y = -, hr[0] == hr[-1] max_score([fake_closed]) < closed_path.score --> closed_path.verity = true
                         "true_open"    : [], # y = +, hr[0] != hr[-1] max_score([true_open]) < open_path.score --> open_path.verity = fake
                         "fake_open"    : []} # y = -, hr[0] != hr[-1]

    # FOR DIVERSITY REWARD CALCULATION
    #past_paths = []

    for episode in range(start + 1, max_episode_num):
        # at the beggining of each go-through, shuffle the train dataset
        i = episode % len_dataset
        x = episode // len_dataset
        if i == 0:
          if random_state:
            env.input = env.input.sample(frac = 1, random_state = random_state)
          else:
            env.input = env.input.sample(frac = 1)

        hr_states, states = env.reset(i)

        log_probs = []
        rewards = []

        for steps in range(max_steps): # [0, 1, 2, 3]
          new_hr_states     = []
          new_states        = []

          for hr_state, state in zip(env.hr_states, env.states):
            current_ent       = hr_state[env.i]
            action, log_prob  = policy_net.get_action(state, env.action_space, current_ent, env.kg)

            new_hr_state, new_state, reward = env.step(hr_state, state, action)

            # collect the updated states
            new_hr_states.append(new_hr_state)
            new_states.append(new_state)

            # for the Policy Network
            # collect log_prob and reward for 1 step
            log_probs.append(log_prob)
            # Give reward at the end of the episode
            if max_steps - steps == 1:
              rewards.append(reward)
            else:
              rewards.append(0)

          # for the environment
          # reset index, update paths
          env.i               += 2
          env.hr_states       = new_hr_states
          env.states          = new_states


        if reward >= 1:
          env.stats["Target found"] += 1

        list_log_probs = [log_prob.item() for log_prob in log_probs]
        if env.y == "+":
          if new_hr_state[0] == new_hr_state[env.i]:
            confidence_scores["true_closed"].append(sum(list_log_probs))
          elif new_hr_state[0] != new_hr_state[env.i]: #max_score([true_open]) < open_path.score --> open_path.verity = fake
            confidence_scores["true_open"].append(sum(list_log_probs)) 
        if env.y == "-":
          if new_hr_state[0] == new_hr_state[env.i]: # y = -, hr[0] == hr[-1] max_score([fake_closed]) < closed_path.score --> closed_path.verity = true
            confidence_scores["fake_closed"].append(sum(list_log_probs))
          elif new_hr_state[0] != new_hr_state[env.i]:
            confidence_scores["fake_open"].append(sum(list_log_probs))

        # stop training classifier with useless paths
        if sum(rewards) > 0:
          path = torch.tensor(new_state.flatten()).float()
          pred_y = env.classification_model(path)
          true_y = torch.tensor(np.array([float(env.y == "+")]))
          pred_y = pred_y.float()
          true_y = true_y.float()
          loss = env.loss_function(pred_y, true_y)
          loss = loss.float()
          env.classification_model.zero_grad()
          loss.backward()

        #reward, past_paths = diversity_reward(new_state[3:], past_paths)
        for i in range(4-len(rewards)):
          rewards.append(0)
                
        update_policy(policy_net, rewards, log_probs)
        all_rewards.append(np.sum(rewards)) # What to do with this?

        
        if (episode + 1) % 1000 == 0:
          mean_reward = np.mean(all_rewards[-1000:])
          print(episode, ":", mean_reward)
          avg_rewards.append(np.mean(all_rewards[-1000:]))
        
        if (episode + 1) % 10000 == 0:
          if not os.path.exists("models"):
            os.makedirs("models")
          net_name = str(episode+1) + "_ComplEx_policy_net"
          torch.save(policy_net, "models/" + net_name + ".pt")
          with open("log.txt", "a") as file:
            file.write(net_name + "\tMean reward for last 1K episodes: " + str(mean_reward) + "\n")
        
        
    print(env.stats)
    plt.plot(avg_rewards)
    plt.legend(['reward'])
    plt.xlabel('Episode')
    plt.title("avg rew, path len")
    plt.show()

    return policy_net, env.classification_model, confidence_scores







def clas_tag(env, clas_verdict):
  if env.y == "-":
    if clas_verdict:
      return "False positive"  
    else:
      return "True negative"
  elif env.y == "+":
    if clas_verdict:
      return "True positive"
    else:
      return "False negative"

def path_tag(path_success, path_complete):
  if path_success:
    if path_complete:
      return "True claim confirmed"
    else:
      return "False claim busted"
  else:
    if path_complete:
      return "False claim reproduced"
    else:
      return "Nothing accomplished"

def exclude_already_chosen_entities(new_ents, top_ent, scores):
  indices = [i for i, x in enumerate(new_ents) if x == top_ent]
  for i in indices:
    scores[i] = 0
  return scores

def exclude_already_chosen_paths(new_hr_paths, top_hr_path, scores):
  indices = [i for i, x in enumerate(new_hr_paths) if x == top_hr_path]
  for i in indices:
    scores[i] = 0
  return scores

def test_model(path_to_model, policy_net, classification_model, b, max_steps):

    policy_net.train = False
    number_of_successful_episodes = 0

    size_observation_space = 440 #(len(list(relevant_aliases.keys())))
    env = Environment_test("test.csv", path_to_model)
    size_action_space = len(env.action_space)
    
    max_episode_num = env.input.shape[0]
    env.stats["Number of episodes:"] = max_episode_num
    env.stats["Size of action space:"] = size_action_space
    max_steps = max_steps
    number_of_correct_classifications = 0

    path_results = {"True claim confirmed" : [],
                    "False claim busted" : [],
                    "False claim reproduced" : [],
                    "Nothing accomplished" : []}

    for episode in range(max_episode_num):
      
        hr_states, states = env.reset(i = episode)

        for steps in range(max_steps):
          new_hr_states     = []
          new_states        = []
          scores            = []
          actionsss         = []
          new_ents          = []
          top_hr_states     = []
          top_states        = []

          for hr_state, state in zip(env.hr_states, env.states):
            current_ent     = hr_state[env.i]
            actions, logps  = policy_net.get_actions(state, env.action_space, current_ent, env.kg, b)

            for action, logp in zip(actions, logps):
              new_hr_stater, new_stater, reward, new_enter, scorer = env.step(hr_state, state, action)

              # collect new states

              new_hr_states.extend(new_hr_stater)
              new_states.extend(new_stater)
              new_ents.extend(new_enter)

              # calculate heuristic
              scorer = np.asarray(scorer)
              scorer **= steps
              scorer += np.e**logp.item()
              #for i, sc in enumerate(scorer):
              #  scorer[i] = np.e** logp.item() + sc**steps
              
              scores.extend(scorer)
            
          scores = np.asarray(scores)
          ### Voting system ###
          scores_for_the_vote = []
          ### Voting system ###

          #kärpimine
          for i in range(b):
            top_score_index = scores.argmax()
            top_ent = new_ents[top_score_index]
            top_hr_state = new_hr_states[top_score_index]
            #top_hr_state[-1] = i
            top_hr_states.append(top_hr_state)
            top_states.append(new_states[top_score_index])

            ### Voting system #### using heuristics to vote
            scores_for_the_vote.append(scores[top_score_index]) 
            ### Voting system #### using heuristics to vote
            #scores[top_score_index] = 0
            
            # Previously, if an entity was chosen to extend one of the paths
            # that entity was excluded from being chosen for any other path
            # That was done to keep the agent from resulting in multiple
            # variations of the path "correct_relation and self_loops"
            # To facilitate the the voting mechanism, now only the specific
            # instance of a chosen entity is excluded so as not to pick the
            # exact same path again for another beam
            #scores[top_score_index] = 0
            scores = exclude_already_chosen_paths(new_hr_states, top_hr_state, scores)

          # for the environment
          # reset index, update paths
          env.i               += 2
          env.hr_states       = top_hr_states
          env.states          = top_states

        episode_success_flag = 0
        # statistics
        ### VOTING SYSTEM ###
        ents = []
        cnts = []
        ### VOTING SYSTEM ###


        for path, voting_score in zip(top_hr_states, scores_for_the_vote): # using heuristics to vote
          finish_ent = path[env.i]
          
          ### voting system ###
          if finish_ent in ents:
            iii = ents.index(finish_ent)
            cnts[iii] += 1
            #cnts[iii] += voting_score ### voting system, using heuristics to vote
          else:
            ents.append(finish_ent)
            cnts.append(1)
            #cnts.append(voting_score) ### voting system, using heuristics to vote
          ### voting system ###

          claim_targ = path[0]
          path_complete = finish_ent == claim_targ
          path_success = finish_ent in env.actual_target
          #if path_success:
          #  episode_success_flag = 1
          tag = path_tag(path_success, path_complete)
          path_results[tag].append((episode, path))
        
        ### VOTING SYSTEM ###
        ggg = cnts.index(max(cnts))
        guess = ents[ggg]
        episode_success_flag = int(guess in env.actual_target)
        ### VOTING SYSTEM ###

        number_of_successful_episodes += episode_success_flag

    print("Path results:")
    for key in path_results:
      print(key + ":", len(path_results[key]))

    accuracy = number_of_successful_episodes / max_episode_num
    return path_results, accuracy

In [ ]:
#os.chdir("/content/drive/MyDrive/Colab Notebooks/thesis/NELL-995")
for i, o in enumerate(os.listdir("./models")):
  print(i, o)

FileNotFoundError: ignored

In [ ]:
policy_net_path = os.listdir("./models")[19]
policy_net = torch.load("./models/"+policy_net_path)
start = int(policy_net_path.split("_")[0])
start

200000

In [ ]:
path_results_ = []
max_steps = 3
start = start
max_episode_num = 5*10**
b = 1
rs = 1
ms = 5
policy_net, classification_model, confidence_scores = train_model("ComplEx", max_episode_num, max_steps, random_state = rs, manual_seed = ms, policy_net = policy_net, start = start)
bees = [3, 5, 10]
for b in bees:
  path_results, accuracy = test_model("ComplEx", policy_net, classification_model, b, max_steps)
  print("Beam number:", b)
  print("Vote accrcy:", accuracy)
torch.save(policy_net, "300k_top3Action_bestEntity.pt")

['entity2id.txt', 'entity2vec.bern', 'evaluate_task.txt', 'raw.kb', 'relation2id.txt', 'kb_env_rl.txt', 'ranked_relations.csv', 'relation2vec.bern', 'whole_sort_test.txt', 'whole_sort_val.txt', 'whole_train_pos_neg.txt', 'dict_kg.pkl', 'application.log', 'test.csv', 'train.csv', 'val.csv', 'kg_triples.csv', 'ComplEx', 'models', '100k_top3Action_bestEntity.pt', 'path_results.pkl', 'accuracy.pkl', 'log.txt', 'tasks', 'train_pos.pkl', 'action_space.pkl', 'kg.csv', 'kg_triples.pkl']


KeyError: ignored

In [ ]:
# algoritmi tuleb timmida niimoodi, et kui hinnatakse pathi unikaalsust, siis selfLoope arvesse ei võeta
# nt [Temake, selfLoop, Temake, ütles, Seda] == [Temake, ütles, Seda, selfLoop]
for e in path_results["Nothing accomplished"]:
  print(e)

Streaming output truncated to the last 5000 lines.
(1493, ['sportsleague_mlb', 'teamplaysinleague', 'sportsteam_l_a__kings', 'teamplaysincity', 'county_los_angeles_county', 'selfLoop', 'county_los_angeles_county', 'istallerthan', 'publication_people_', 0, 0])
(1494, ['sport_boxing', 'athleteplayssport', 'athlete_asdrubal_cabrera', 'selfLoop', 'athlete_asdrubal_cabrera', 'athleteflyouttosportsteamposition', 'sportsteamposition_center', 'selfLoop', 'sportsteamposition_center', 0, 0])
(1494, ['sport_boxing', 'athleteplayssport', 'athlete_asdrubal_cabrera', 'athleteflyouttosportsteamposition', 'sportsteamposition_center', 'selfLoop', 'sportsteamposition_center', 'selfLoop', 'sportsteamposition_center', 0, 0])
(1494, ['sport_boxing', 'athleteplayssport', 'athlete_asdrubal_cabrera', 'selfLoop', 'athlete_asdrubal_cabrera', 'selfLoop', 'athlete_asdrubal_cabrera', 'selfLoop', 'athlete_asdrubal_cabrera', 0, 0])
(1495, ['sportsleague_mlb', 'teamplaysinleague', 'sportsteam_texas_christian', 'selfL

KeyboardInterrupt: ignored

In [ ]:
#RUN ME
path_results_ = []
accuracy_ = []
max_steps = 3
rs = 1
ms = 5
#policy_net = torch.load("100k_top3Action_bestEntity.pt")
for b in [3]:#, 5, 10]:
  print(b)
  path_results, accuracy = test_model("ComplEx", policy_net, None, b, max_steps)
  path_results_.append(path_results)
  accuracy_.append(accuracy)
  print(path_results)
  print(accuracy)

pickle.dump(path_results_, open("path_results.pkl", "wb"))
pickle.dump(accuracy_, open("accuracy.pkl", "wb"))



3


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
for accuracy, path_results in zip(accuracy_, path_results_):
  keys = list(path_results.keys())
  amounts = [len(path_results[key]) for key in keys]
  overall_acc = sum(amounts[:2]) / sum(amounts[2:])
  print("overall_acc:", round(overall_acc,3))
  print("acc:", round(accuracy,3))
  print()

overall_acc: 0.225
acc: 0.246

overall_acc: 0.141
acc: 0.259

overall_acc: 0.092
acc: 0.233



In [ ]:
print(amounts[:2])

[[(31, ['company_pbs', 'agentbelongstoorganization', 'televisionstation_ktsc_tv', 'subpartof', 'company_pbs', 'selfLoop', 'company_pbs', 'selfLoop', 'company_pbs', 0, 0]), (31, ['company_pbs', 'agentbelongstoorganization', 'televisionstation_ktsc_tv', 'agentcollaborateswithagent', 'company_pbs', 'selfLoop', 'company_pbs', 'selfLoop', 'company_pbs', 0, 0]), (31, ['company_pbs', 'agentbelongstoorganization', 'televisionstation_ktsc_tv', 'selfLoop', 'televisionstation_ktsc_tv', 'agentcollaborateswithagent', 'company_pbs', 'selfLoop', 'company_pbs', 0, 0]), (42, ['stadiumoreventvenue_united_center', 'athletehomestadium', 'athlete_kirk_hinrich', 'athleteledsportsteam', 'sportsteam_chicago_bulls', 'teamhomestadium', 'stadiumoreventvenue_united_center', 'selfLoop', 'stadiumoreventvenue_united_center', 0, 0]), (52, ['ceo_john_chambers', 'organizationhiredperson', 'biotechcompany_cisco_systems_inc_', 'organizationterminatedperson', 'ceo_john_chambers', 'selfLoop', 'ceo_john_chambers', 'selfLoop

In [ ]:
b = 3
rs = 1
ms = 5
policy_net, classification_model, confidence_scores = train_model("ComplEx", max_episode_num, max_steps, random_state = rs, manual_seed = ms)#,  policy_net = policy_net, start = start)
path_results = test_model("ComplEx", policy_net, classification_model, b, max_steps)

In [ ]:
torch.save(policy_net, "100k_top3Action_bestEntity.pt")

In [ ]:
path_results, accuracy = path_results
# accuracy on votingu tulemuste täpsus ehk siis mood(path_search(masin_õpe)) = kõik_juhtumid
# path results on kõikide otsingutulemuste kogum
# Ehk siis path results on üleüldine täpsus intelligentse heuristikaga otsingualgoritmile
# Hääletustulemuste täpsust ehk siis demokraatliku otsuse efektiivsusmäär
# Demokraatliku otsuse täpsus - Democratic verdict accuracy

In [ ]:
path_results = test_model("ComplEx", policy_net, classification_model, b, max_steps)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Path results:
True claim confirmed: 252
False claim busted: 1153
False claim reproduced: 23
Nothing accomplished: 5433


In [ ]:
path_results

In [ ]:
!ls

application.log    kg.csv		 train_pos.pkl
ComplEx		   ranked_relations.csv  val.csv
dict_kg.pkl	   raw.kb		 whole_sort_test.txt
entity2id.txt	   relation2id.txt	 whole_sort_val.txt
entity2vec.bern    relation2vec.bern	 whole_train_pos_neg.txt
evaluate_task.txt  test.csv
kb_env_rl.txt	   train.csv


In [ ]:
bees = [10]
max_steps = 3

path_resultses = []
#policy_net = torch.load("100k_top3Action_bestEntity.pt")
path = "ComplEx"
for b in bees:
  print("100K ComplEx, top", b, "actions and entities")
  path_results, accuracy = test_model(path, policy_net, None, b, max_steps)
  print("accuracy:", accuracy)
  path_resultses.append(path_results)

100K ComplEx, top 10 actions and entities


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Path results:
True claim confirmed: 989
False claim busted: 1351
False claim reproduced: 73
Nothing accomplished: 2877
accuracy: 0.7674858223062382


In [ ]:
bees = [10]
max_steps = 3

path_resultses = []
#policy_net = torch.load("100k_top3Action_bestEntity.pt")
print("Vote = 1 for everyone./nLast one was path final ent complex_score + prob for final action")
path = "ComplEx"
for b in bees:
  print("100K ComplEx, top", b, "actions and entities")
  path_results, accuracy = test_model(path, policy_net, None, b, max_steps)
  print("accuracy:", accuracy)
  path_resultses.append(path_results)

In [ ]:
path_results, accuracy = test_model(path, policy_net, None, b, max_steps)
print("accuracy:", accuracy)
path_resultses.append(path_results)

NameError: ignored

In [ ]:
b = 10
path_results = path_resultses[2]
print("b:", b)
for key in path_results:
  print(key)
  for row in path_results[key]:
    print(row)
print("______________________________________________________________________________________________________________________________")

b: 10
True claim confirmed
(1, ['/m/0h7x', 'nationality', '/m/0c73z', 'place_of_death', '/m/0fhp9', 'contains_inv', '/m/0h7x', 'selfLoop', '/m/0h7x', 0, 0])
(1, ['/m/0h7x', 'nationality', '/m/0c73z', 'place_of_death', '/m/0fhp9', 'selfLoop', '/m/0fhp9', 'country', '/m/0h7x', 0, 0])
(1, ['/m/0h7x', 'nationality', '/m/0c73z', 'place_of_death', '/m/0fhp9', 'selfLoop', '/m/0fhp9', 'administrative_parent', '/m/0h7x', 0, 0])
(1, ['/m/0h7x', 'nationality', '/m/0c73z', 'place_of_death', '/m/0fhp9', 'selfLoop', '/m/0fhp9', 'capital_inv', '/m/0h7x', 0, 0])
(1, ['/m/0h7x', 'nationality', '/m/0c73z', 'religion', '/m/0c8wxp', 'selfLoop', '/m/0c8wxp', 'religion_inv', '/m/0h7x', 0, 0])
(3, ['/m/09c7w0', 'nationality', '/m/0h1nt', 'location', '/m/0mp3l', 'country', '/m/09c7w0', 'selfLoop', '/m/09c7w0', 0, 0])
(3, ['/m/09c7w0', 'nationality', '/m/0h1nt', 'location', '/m/0mp3l', 'selfLoop', '/m/0mp3l', 'country', '/m/09c7w0', 0, 0])
(3, ['/m/09c7w0', 'nationality', '/m/0h1nt', 'film', '/m/02qr3k8', 'sel

In [ ]:
path_results_3 = path_resultses[0]
key = "False claim busted"
results = path_results_3[key]
for result in results:
  print(result)

In [ ]:
bees = [3, 5, 10]
for b, path_results in zip(bees, path_resultses):
  print(b)
  for key in path_results:
    print(key)
    counts = {}
    for p in path_results[key]:
      id, path = p
      confidence_rank = path[-1]
      if confidence_rank in counts:
        counts[confidence_rank] += 1
      else: counts[confidence_rank] = 1
    keys = list(counts.keys())
    keys.sort()
    total = 0
    for key in keys:
      total += counts[key]
    for key in keys:
      proportion = round(counts[key] / total, 2)
      print(key, proportion)
    print()

3
True claim confirmed
0 0.73
1 0.11
2 0.16

False claim busted
0 0.74
1 0.17
2 0.09

False claim reproduced
1 0.71
2 0.29

Nothing accomplished
0 0.16
1 0.4
2 0.44

5
True claim confirmed
0 0.8
1 0.08
2 0.03
3 0.03
4 0.05

False claim busted
0 0.68
1 0.17
2 0.04
3 0.06
4 0.06

False claim reproduced
0 0.14
1 0.21
2 0.36
3 0.14
4 0.14

Nothing accomplished
0 0.06
1 0.22
2 0.24
3 0.24
4 0.24

10
True claim confirmed
0 0.73
1 0.11
2 0.05
4 0.02
5 0.02
7 0.02
8 0.01
9 0.02

False claim busted
0 0.55
1 0.14
2 0.08
3 0.03
4 0.06
5 0.02
6 0.03
7 0.02
8 0.06
9 0.03

False claim reproduced
1 0.31
2 0.06
4 0.06
5 0.12
6 0.19
7 0.19
9 0.06

Nothing accomplished
0 0.03
1 0.09
2 0.1
3 0.11
4 0.11
5 0.11
6 0.11
7 0.11
8 0.11
9 0.11



In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
def split_embedding(embedding):
  real = embedding[:20]
  imag = embedding[20:]
  embedding = (real, imag)
  return embedding

def cosine_similarity(one, two):    
  return dot(one, two) / (norm(one)*norm(two))

def embeddings_for_index(i):
  triple = kg.iloc[i]
  source, relation, target = triple
  source_e, target_e = model.get_embeddings([source, target])
  relation_e = model.get_embeddings(relation, "relation")
  return source_e, relation_e, target_e

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def complexify(input_vector):
  output_vector = []
  for real, imag in zip(input_vector[:20], input_vector[20:]):
    comp = complex(real, imag)
    output_vector.append(comp)
  output_vector = np.asarray(output_vector)
  return output_vector

def dot_product(source, relation, target):
  target = target.conjugate()
  dot_product = 0
  for u, a, e in zip(source, relation, target):
    dot_product += u * a * e
  return dot_product.real

def embeddings_for_index(i):
  triple = kg.iloc[i]
  source, relation, target = triple
  source_e, target_e = model.get_embeddings([source, target])
  relation_e = model.get_embeddings(relation, "relation")
  return source_e, target_e, relation_e

In [ ]:
asp =pickle.load(open("action_space.pkl", "rb"))
asp.remove("major_field_of_study_inv")
#asp.append("languages_inv")
pickle.dump(asp, open("action_space.pkl", "wb"))

In [ ]:
resultant_path_keys = ["True claim confirmed", "False claim busted"]
ids = []
for rpk in resultant_path_keys:
  for p in path_results[rpk]:
    id, _ = p
    ids.append(id)
ids.sort()
failed_indices = np.setdiff1d([*range(168)], ids)
failed_indices

array([  2,  14,  16,  19,  23,  25,  27,  28,  31,  32,  33,  42,  45,
        49,  50,  55,  58,  59,  60,  63,  64,  69,  70,  72,  74,  75,
        77,  83,  85,  87,  94,  97, 100, 101, 104, 106, 107, 115, 116,
       117, 120, 121, 122, 123, 124, 126, 131, 132, 133, 141, 142, 147,
       154, 157, 158, 161, 162, 165])

In [ ]:
resultant_path_keys = ["True claim confirmed", "False claim busted"]
relation_indices = [3, 5, 7]
the_models_action_choices = []

for path_results in path_resultses:
  action_choices = []
  for rp_key in resultant_path_keys:
    a_list_of_tuples_id_and_path = path_results[rp_key]
    for tup in a_list_of_tuples_id_and_path:
      id, path = tup
      relations_of_a_path = []
      for ri in relation_indices:
        a_relation = path[ri]
        if a_relation != "selfLoop":
          relations_of_a_path.append(a_relation)
      action_choices.append(relations_of_a_path)
  the_models_action_choices.append(action_choices)
"""
for path_results in path_resultses:
  for result in path_results:
    print(result)
    for thing in path_results[result]:
      print(thing)
    
  print()
"""

'\nfor path_results in path_resultses:\n  for result in path_results:\n    print(result)\n    for thing in path_results[result]:\n      print(thing)\n    \n  print()\n'

In [ ]:
for acs in the_models_action_choices:
  acs_singles = []
  count = []
  for roap in acs:
    if roap in acs_singles:
      count[acs_singles.index(roap)] += 1
    else:
      acs_singles.append(roap)
      count.append(1)
  for rOAP, cOUNT in zip(acs_singles, count):
    print(rOAP, cOUNT)
  print()

['place_of_birth'] 51
['location'] 20
['type_of_union', 'location_of_ceremony'] 4
['location', 'country'] 1
['place_of_birth', 'country'] 1
['group_inv', 'location'] 1
['place_of_birth', 'contains_inv'] 1
['place_of_birth', 'second_level_divisions_inv'] 1
['award_nominee_inv', 'place_of_birth'] 1

['place_of_birth'] 51
['award_nominee_inv', 'place_of_birth'] 2
['location'] 21
['type_of_union', 'location_of_ceremony'] 9
['artist_inv', 'citytown'] 4
['place_of_birth', 'contains_inv'] 1
['group_inv', 'location'] 3
['place_of_birth', 'second_level_divisions_inv'] 1
['artist_inv', 'place_founded'] 1

['place_of_birth'] 51
['award_nominee_inv', 'place_of_birth'] 3
['location'] 21
['group_inv', 'location'] 12
['group_inv', 'place_of_birth'] 3
['type_of_union', 'location_of_ceremony'] 9
['place_of_death'] 1
['place_of_birth', 'contains_inv'] 2
['group_inv', 'nationality'] 2
['artist_inv', 'citytown'] 3
['peers_inv', 'location'] 1
['award_winner', 'place_of_birth'] 1
['nationality'] 1
['place_o

In [ ]:
max_steps = 3
iterations = 300
b = 3
rs = 1
ms = 5
paths = ["ComplEx"] # ["TransE"], ["TransE", "ComplEx"], ["ComplEx"]
for path in paths:
  print(path, "b:", b)
  policy_net, classification_model, confidence_scores = train_model(path, iterations, max_steps, random_state = rs, manual_seed = ms, policy_net = policy_net)
  #path_results = test_model(path, policy_net, classification_model, b, max_steps)
  #print()
  torch.save(policy_net, "300k_top3Action_bestEntity.pt")

bees = [3, 5, 10]

path_resultses = []
print("300K ComplEx, top 3 actions, best entity")
for b in bees:
  path_results, accuracy = test_model(path, policy_net, classification_model, b, max_steps)
  print(b, ":", accuracy)
  path_resultses.append(path_results)

pickle.dump(path_resultses, open("path_resultses_300k_3_5_10.pkl", "wb"))

In [ ]:
bees = [3, 5, 10]

path_resultses = []
print("300K ComplEx, top 3 actions, best entity")
for b in bees:
  path_results, accuracy = test_model(path, policy_net, classification_model, b, max_steps)
  print(b, ":", accuracy)
  path_resultses.append(path_results)

pickle.dump(path_resultses, open("path_resultses_300k_3_5_10.pkl", "wb"))

In [ ]:
for path_results in path_resultses:
  for pr in path_results:
    print(pr)
    for p in path_results[pr]:
      print(p)
  print()

True claim confirmed
(6, ['/m/02dtg', 'origin', '/m/01kp_1t', 'place_of_birth', '/m/02dtg', 'selfLoop', '/m/02dtg', 'selfLoop', '/m/02dtg', 0, 0])
(8, ['/m/094jv', 'origin', '/m/0dbb3', 'location', '/m/094jv', 'selfLoop', '/m/094jv', 'selfLoop', '/m/094jv', 0, 1])
(15, ['/m/0ftvz', 'origin', '/m/0837ql', 'place_of_birth', '/m/0ftvz', 'selfLoop', '/m/0ftvz', 'selfLoop', '/m/0ftvz', 0, 1])
(18, ['/m/01sn3', 'origin', '/m/016s0m', 'place_of_birth', '/m/01sn3', 'selfLoop', '/m/01sn3', 'selfLoop', '/m/01sn3', 0, 0])
(20, ['/m/06wxw', 'origin', '/m/016pns', 'location', '/m/06wxw', 'selfLoop', '/m/06wxw', 'selfLoop', '/m/06wxw', 0, 1])
(22, ['/m/0cr3d', 'origin', '/m/01n44c', 'place_of_birth', '/m/0cr3d', 'selfLoop', '/m/0cr3d', 'selfLoop', '/m/0cr3d', 0, 0])
(38, ['/m/04jpl', 'origin', '/m/0qf11', 'type_of_union', '/m/04ztj', 'selfLoop', '/m/04ztj', 'location_of_ceremony', '/m/04jpl', 0, 2])
(40, ['/m/0f2tj', 'origin', '/m/04n32', 'place_of_birth', '/m/0f2tj', 'selfLoop', '/m/0f2tj', 'selfLo

4

In [ ]:
## READING KNOWLEDGE GRAPH ###

def train_pos():

  t = {}

  with open("train_pos") as file:
    for l in file:
      l = l.strip().split("\t")
      source = l[0]
      target = l[1]
      if source in t:
        t[source].append(target)
      else: t[source] = [target]
  
  return t

def read_triples():


  t = {"e0":[], "r":[], "e1":[]}

  with open("graph.txt") as file:
    for l in file:
      l = l.strip().split("\t")
      t["e0"].append(l[0])
      t["r"].append(l[1].split("/")[-1]) # SHORTEN THE RELATION WITH .split("/")[-1]
      t["e1"].append(l[2])

  with open("train_pos") as file:
    for l in file:
      l = l.strip().split("\t")
      t["e0"].append(l[0])
      t["e1"].append(l[1])
      t["r"].append(l[2].split("/")[-1]) # SHORTEN THE RELATION WITH .split("/")[-1]

  return pd.DataFrame(t)

def read_kg_triples():
  t = {}

  with open("graph.txt") as file:
    all_entities = set()
    for l in file:
      l = l.strip().split("\t")
      
      e0 = l[0]
      r = l[1].split("/")[-1]
      e1 = l[2]

      if e0 == e1:
        continue

      #Collect all entities, use to generate self loops
      all_entities.add(e0)
      all_entities.add(e1)

      e0key = e0+r

      if e0key in t:
        t[e0key].append(e1)
      else:
        t[e0key] = [e1]
      
    all_entities = list(all_entities)
    for e in all_entities:
      t[e+"selfLoop"] = [e]

  return t

#¤¤ READING SOURCE AND TARGET PAIRS FOR TRAINING ¤¤#
def read_input():
  p = {"es":[], "rr": [], "et":[], "y":[]}
  root_relation = os.getcwd().split("@")[-1]  # Kas kaasata ka root_relation + "_inv" ehk tagurpidine suhe
  root_relation

  with open("train.pairs") as file:
    for l in file:
      l = l.replace("thing$", "/")
      l = l.replace(": ", ",")
      l = l.strip().split(",")
      p["es"].append(l[0].replace("_", "/", 1))
      p["rr"].append(root_relation)
      p["et"].append(l[1].replace("_", "/", 1))
      p["y"].append(l[2])

  return pd.DataFrame(p)